In [1]:
# LIBRARIES
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import load_model


In [2]:
np.random.seed(1234)
pd.set_option('display.max_rows', None)

In [3]:
def generate_observation_sigma(space_obs_frac=0.5):
    """
    Draws a standard deviation of noise in light curve points from a "true" value provided in synthetic light curve.
    Noise sigma is drawn from bimodal distribution taking into account contributions from space based and earth based
    observations which have different levels of stochastic noise.

    :param space_obs_frac: ratio between earth based and space based observations
    :return: float; standard deviation of the light curve noise
    """
    earth_based_sigma = 4e-3
    space_based_sigma = 2e-4
    sigma = np.random.choice([earth_based_sigma, space_based_sigma], p=[1-space_obs_frac, space_obs_frac])
    return np.random.rayleigh(sigma)

def stochastic_noise_generator(curve):
    """
    Introduces gaussian noise into synthetic observation provided in `curve`.

    :param curve: numpy.array; normalized light curve
    :return: Tuple(numpy.array, float); normalized light curve with added noise, standard deviation of observations
    """
    sigma = generate_observation_sigma()
    return np.random.normal(curve, sigma), np.full(curve.shape, sigma)

In [4]:
data = pd.read_pickle("detached_all_parameters.pkl").reset_index()
data.head()

,index,id,curve,primary__t_eff,secondary__t_eff,inclination,mass_ratio,primary__surface_potential,secondary__surface_potential,t1_t2,filter,critical_surface_potential,primary__equivalent_radius,secondary__equivalent_radius,primary__filling_factor,secondary__filling_factor
0,0,38,"[0.6055271686415179, 0.9842041250556204, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_U,15.09104,0.009996,0.009996,-145.333979,-1502.830354
1,1,38,"[0.608985656265516, 0.9846965713304289, 0.9998...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_B,15.09104,0.009996,0.009996,-145.333979,-1502.830354
2,2,38,"[0.6189025614226916, 0.9837351924934223, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_V,15.09104,0.009996,0.009996,-145.333979,-1502.830354
3,3,38,"[0.6292771409565273, 0.9832675811171884, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_R,15.09104,0.009996,0.009996,-145.333979,-1502.830354
4,4,38,"[0.6543378609145588, 0.9835188424579704, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_I,15.09104,0.009996,0.009996,-145.333979,-1502.830354


In [13]:
cols = ["inclination",
        "mass_ratio",
        "primary__surface_potential",
        "secondary__surface_potential",
        "t1_t2"]
inc_set, q_set, omega1_set, omega2_set, t1_t2_set = [], [], [], [], []

for col in data:
    if col == "inclination":
        inc_set = sorted(list(set(data[col])))
    if col == "mass_ratio":
        q_set = sorted(list(set(data[col])))
    if col == "primary__surface_potential":
        omega1_set = sorted(list(set(data[col])))
    if col == "secondary__surface_potential":
        omega2_set = sorted(list(set(data[col])))
    if col == "t1_t2":
        t1_t2_set = sorted(list(set(data[col])))

In [16]:
det_inc = load_model("models/detached_inclination.hdf5")        # inclination
det_mass_ratio = load_model("models/detached_mass_ratio.hdf5")  # mass ratio
det_prim_potent = load_model("models/detached_pSP.hdf5")        # primary surface potential
det_sec_potent = load_model("models/detached_sSP.hdf5")         # secondary surface potential
det_temp_ratio = load_model("models/detached_t1_t2.hdf5")       # temperature ratio

In [17]:
data_sample = data.sample(n=100)

In [18]:
X = []
for row in data_sample["curve"]:
    X.append(row)
X=np.array(X)

# y_inc = np.array(data_sample["inclination"])
# y_mass_ratio = np.array(data_sample["mass_ratio"])
# y_psp = np.array(data_sample["primary__surface_potential"])
# y_ssp = np.array(data_sample["secondary__surface_potential"])
# y_t2_t1 = np.array(data_sample["t1_t2"])

pred_inc = det_inc.predict(X).flatten()
pred_q = det_mass_ratio.predict(X).flatten()
pred_pp = det_prim_potent.predict(X).flatten()
pred_sp = det_sec_potent.predict(X).flatten()
pred_tr = det_temp_ratio.predict(X).flatten()

preds = pd.DataFrame()
preds['pred_inc']=pd.Series(pred_inc)
preds['pred_q']=pd.Series(pred_q)
preds['pred_omega1']=pd.Series(pred_pp)
preds['pred_omega2']=pd.Series(pred_sp)
preds['pred_t1_t2']=pd.Series(pred_tr)

preds.head()

,pred_inc,pred_q,pred_omega1,pred_omega2,pred_t1_t2
0,1.343030,2.019274,51.956097,10.617927,8.109618
1,1.285049,1.118809,6.859527,5.204861,2.427836
2,1.413126,2.107352,9.943995,5.053971,2.125901
3,1.246710,1.394126,11.324502,7.799589,2.606561
4,1.169418,1.015253,2.610128,23.238480,1.808659


In [97]:
def find_num(number, s):
    for i in range(len(s)-1):
        if number < s[i] and (i != len(s)-1):
            return s[i]
        elif number > s[i] and (i != len(s)-1):
            continue
        else:
            return max(s)


In [98]:
new_preds_inc = []
for i in preds["pred_inc"]:
    new_preds_inc.append(find_num(i, inc_set))

In [99]:
new_preds_q = []
for i in preds["pred_q"]:
    new_preds_q.append(find_num(i, q_set))

In [100]:
new_preds_omega1 = []
for i in preds["pred_omega1"]:
    new_preds_omega1.append(find_num(i, omega1_set))

In [101]:
new_preds_omega2 = []
for i in preds["pred_omega2"]:
    new_preds_omega2.append(find_num(i, omega2_set))

In [102]:
new_preds_t1_t2 = []
for i in preds["pred_t1_t2"]:
    new_preds_t1_t2.append(find_num(i, t1_t2_set))

In [103]:
new_preds = pd.DataFrame()
new_preds['pred_inc']=pd.Series(new_preds_inc)
new_preds['pred_q']=pd.Series(new_preds_q)
new_preds['pred_omega1']=pd.Series(new_preds_omega1)
new_preds['pred_omega2']=pd.Series(new_preds_omega2)
new_preds['pred_t1_t2']=pd.Series(new_preds_t1_t2)

In [105]:
new_preds.head()

,pred_inc,pred_q,pred_omega1,pred_omega2,pred_t1_t2
0,1.348982,2.500000,100.100050,10.630533,8.750000
1,1.285578,1.250000,6.897109,5.240386,2.500000
2,1.413717,2.500000,10.201022,5.100484,2.142857
3,1.256637,1.428571,11.415168,7.869121,2.666667
4,1.169551,1.111111,2.624005,24.217839,1.875000


In [106]:
preds.head()

,pred_inc,pred_q,pred_omega1,pred_omega2,pred_t1_t2
0,1.343030,2.019274,51.956097,10.617927,8.109618
1,1.285049,1.118809,6.859527,5.204861,2.427836
2,1.413126,2.107352,9.943995,5.053971,2.125901
3,1.246710,1.394126,11.324502,7.799589,2.606561
4,1.169418,1.015253,2.610128,23.238480,1.808659
